In [1]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [2]:
size = 512
channels = 1  #input image format
input_img = Input(shape=(size, size, channels))
#print(input_img)

In [3]:
# Encoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)

In [4]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
autoencoder.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 256, 256, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 128, 128, 16)   │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128, 128, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 64, 64, 16)     │         2,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d (UpSampling2D)    │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_1 (UpSampling2D)  │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 256, 256, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_2 (UpSampling2D)  │ (None, 512, 512, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 512, 512, 1)    │           577 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 50,209 (196.13 KB)

 Trainable params: 49,985 (195.25 KB)

 Non-trainable params: 224 (896.00 B)

In [5]:
def preprocess_image(image):
    #resize image so they're same format
    image = tf.image.resize(image, (size, size), method='lanczos5')
    return image


datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image,
    validation_split=0.15
)


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,  # Rotates the images by up to 10 degrees
    width_shift_range=0.1,  # Shifts the image width by a maximum of 10%
    height_shift_range=0.1,  # Shifts the image height by a maximum of 10%
    shear_range=0.1,  # Shear angle in counter-clockwise direction
    zoom_range=0.1,  # Random zoom
    horizontal_flip=True,  # Randomly flip inputs horizontally
    fill_mode='nearest',  # Strategy to fill newly created pixels
    preprocessing_function=preprocess_image,
    validation_split=0.2
)

In [6]:
import os
os.getcwd()

'/Users/yong/Desktop/thesis'

In [7]:
train_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input',
    subset='validation'
)

Found 71 images belonging to 5 classes.
Found 11 images belonging to 5 classes.


In [8]:
# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Now include the callback in the fit method
autoencoder.fit(
    train_generator,
    epochs=200,
    validation_data=validation_generator,
    callbacks=[early_stopper],
    shuffle = True
)

Epoch 1/200


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 69s 17s/step - loss: 0.2289 - val_loss: 0.2327
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 69s 17s/step - loss: 0.0540 - val_loss: 0.2020
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 80s 20s/step - loss: 0.0089 - val_loss: 0.1529
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 80s 21s/step - loss: 0.0046 - val_loss: 0.1014
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 75s 36s/step - loss: 0.0023 - val_loss: 0.0626
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 81s 22s/step - loss: 0.0013 - val_loss: 0.0406
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 125s 42s/step - loss: 0.0011 - val_loss: 0.0295
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 85s 20s/step - loss: 9.0140e-04 - val_loss: 0.0247
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 84s 21s/step - loss: 8.1467e-04 - val_loss: 0.0230
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 85s 21s/step - loss: 7.5676e-04 - val_loss: 0.0241
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 87s 42s/step - loss: 6.6325e-04 - val_loss: 0.0265
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 99s 23s/step - loss: 6.3113e-04

In [9]:
#save model
autoencoder.save('0505x1x3.keras')